In [1]:
!pip install qiskit
!pip install qiskit ipywidgets
!pip install qiskit-optimization
!pip install pylatexenc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 18.7 MB/s eta 0:00:00
  Created wheel for docplex: fi

In [14]:
# imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.primitives import Sampler
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
from qiskit.circuit import ParameterVector
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from qiskit_algorithms.optimizers import COBYLA
from sklearn.model_selection import train_test_split
import zipfile
import cv2
import seaborn as sns


generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

In [3]:
def process_images(zip_path):
    """
    Extracts and processes images from a ZIP file and returns feature vectors along with
    one-hot encoded labels. The label vectors are automatically created to have the same
    dimensionality as the flattened feature vectors (e.g. 64 if using 8x8 images).

    The filename is used to determine the label:
        - Filenames containing "tri" will have a label [1, 0, 0, ..., 0]
        - Filenames containing "rect" will have a label [0, 1, 0, ..., 0]

    Parameters:
        zip_path (str): Path to the ZIP file.

    Returns:
        np.array: Processed feature vectors.
        np.array: One-hot encoded labels.
    """
    features = []  # List to store flattened image feature vectors
    labels = []    # List to store one-hot encoded label vectors

    # Mapping from substring in the filename to the index that will be set to 1 in the one-hot vector.
    label_mapping = {
        "tri": 0,   # e.g., triangle -> [1, 0, 0, ..., 0]
        "rect": 1   # e.g., square/rectangle -> [0, 1, 0, ..., 0]
    }

    # This will be determined from the first image processed.
    label_dim = None

    with zipfile.ZipFile(zip_path, 'r') as z:
        for file_name in z.namelist():
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                with z.open(file_name) as file:
                    # Read the image using OpenCV
                    file_bytes = file.read()
                    img = cv2.imdecode(np.frombuffer(file_bytes, np.uint8), cv2.IMREAD_GRAYSCALE)

                    if img is None:
                        print(f"Warning: {file_name} could not be read.")
                        continue

                    # Resize the image to 8x8 and normalize pixel values to [0, 1]
                    img = cv2.resize(img, (8, 8))
                    img = img / 255.0

                    # Flatten the image into a 1D array
                    flat_img = img.flatten()
                    features.append(flat_img)

                    # Determine label dimension based on the flattened image
                    if label_dim is None:
                        label_dim = flat_img.size  # e.g., 64 for 8x8 images

                    # Create a one-hot encoded label vector of the same dimension as the feature vector
                    assigned = False
                    for key, index in label_mapping.items():
                        if key in file_name.lower():
                            # Check that the chosen index is valid for this dimension.
                            if index >= label_dim:
                                raise ValueError(f"Label index {index} is out of bounds for a label vector of dimension {label_dim}.")

                            label_vec = np.zeros(label_dim)
                            label_vec[index] = 1

                            labels.append(label_vec)
                            assigned = True
                            break

                    if not assigned:
                        print(f"Warning: {file_name} has no recognized label.")
        return np.array(features), np.array(labels)

In [4]:
def amplitude_encode(amplitudes):
    """
    Normalize the input data and encode it into a quantum state.

    Parameters:
        amplitudes (list or np.array): Input data to be encoded.

    Returns:
        QuantumCircuit: Quantum circuit with the encoded data.
    """
    amplitudes = np.array(amplitudes)
    # Check that the size is a power of 2
    if amplitudes.size & (amplitudes.size - 1) != 0:
        raise ValueError("Input size must be a power of 2.")
    # Normalize the vector
    amplitudes = amplitudes / np.sqrt(np.sum(amplitudes**2))
    num_qubits = int(np.log2(amplitudes.size))
    qc = QuantumCircuit(num_qubits)
    qc.initialize(amplitudes, range(num_qubits))
    return qc

In [5]:
def u_theta(amplitudes, parameters, entanglement, layers):
    """
    Create a quantum neural network (QNN) circuit.
    This circuit encodes the input amplitudes and then applies a parametrized TwoLocal circuit.
    Parameters:
        amplitudes (list or np.array): Input data to be encoded into quantum states.
        parameters (list or np.array): Trainable parameters for the TwoLocal circuit.
        entanglement (str or list): Defines the entanglement pattern between qubits
                                    (e.g., 'linear', 'full', or a custom list).
        layers (int): Number of repetitions (depth) of the TwoLocal circuit.

    Returns:
        QuantumCircuit: A quantum circuit with input encoding and trainable variational layers.

    """
    num_qubits = int(np.log2(len(amplitudes)))

    # Create the parametrized circuit using TwoLocal
    twolocal = TwoLocal(
        num_qubits=num_qubits,
        rotation_blocks=['ry'],
        entanglement_blocks='cx',
        entanglement=entanglement,
        reps=layers  # number of hidden layers
    )

    # Bind parameters to the circuit
    param_dict = dict(zip(twolocal.parameters, parameters))
    twolocal.assign_parameters(param_dict, inplace=True)

    # Compose the feature map and the parametrized circuit
    qnn = QuantumCircuit(num_qubits, num_qubits)
    feature_map = amplitude_encode(amplitudes)
    qnn.compose(feature_map, inplace=True)
    qnn.compose(twolocal, inplace=True)
    # qnn.measure(range(num_qubits), range(num_qubits)) # COMMENT OUT IF USING SAMPLER!

    return qnn

In [6]:
def cost_function(params, features, labels, entanglement, layers):
    """
    Compute the total cost (sum of squared differences) between the measured output
    and the one-hot encoded label over all training examples.
    """
    total_error = 0

    for feature, label in zip(features, labels):
        feature = np.array(feature)
        qnn = u_theta(feature, params, entanglement, layers)

        # Simulate the circuit using Statevector (instead of Sampler)
        statevector = Statevector.from_instruction(qnn)

        # Get the probabilities from the statevector data
        probabilities = np.abs(statevector.data) ** 2  # Square the amplitudes to get probabilities

        total_error += np.sum((np.array(label) - probabilities) ** 2)
        print(f"Feature: {feature}, Label: {label}, Probabilities: {probabilities}, Error: {total_error/len(features)}")

    return total_error / len(features)

In [7]:
def cost_function_sampler(params, features, labels, entanglement, layers):
  """
    Compute the cost function using a quantum circuit with the Sampler.

    This function evaluates the sum of squared differences between the measured
    output probabilities from the quantum circuit and the expected one-hot encoded labels.

    Parameters:
        params (np.array): Trainable parameters for the quantum circuit.
        features (list of np.array): Input data vectors to be encoded into quantum states.
        labels (list of np.array): Target output states (one-hot encoded).
        entanglement (str or list): Defines the entanglement pattern (e.g., 'linear', 'full').
        layers (int): Number of layers in the TwoLocal ansatz.

    Returns:
    float: The mean squared error (MSE) between the output probability distributions
               and the target labels.
    """
  total_error = 0
  sampler = Sampler()

  for feature, label in zip(features, labels):
      feature = np.array(feature)
      qnn = u_theta(feature, params, entanglement, layers)
      qnn.measure(range(qnn.num_qubits), range(qnn.num_qubits))
      result = sampler.run(qnn).result()

      # Create an empty probability vector
      probabilities = np.zeros(len(label))
      # Loop over the measurement outcomes
      for key, value in result.quasi_dists[0].items():
          if isinstance(key, str):
               index = int(key, 2)
          else:
             index = key
          probabilities[index] = value

      total_error += np.sum((np.array(label) - probabilities) ** 2)
      print(f"Feature: {feature}, Label: {label}, Probabilities: {probabilities}, Error: {total_error/len(features)}")

  return total_error / len(features)

In [19]:
def optimization(costfunction, params, features, labels, optimizer, entanglement, layers, color):
    """
    Perform optimization to minimize the cost function for a quantum machine learning model.

    Parameters:
        costfunction (function): The cost function to be minimized. This can use either
                                 statevector simulation or a sampler for probability extraction.
        params (array): Initial parameter values for the variational quantum circuit.
        features (array): The input feature vectors (e.g., quantum-encoded image data).
        labels (array): The target output states (e.g., quantum state representations of transformed images).
        optimizer (str): The optimization algorithm to use. Examples include:
                         - "COBYLA" (Constrained Optimization By Linear Approximations)
                         - "L-BFGS-B" (Limited-memory Broyden–Fletcher–Goldfarb–Shanno with Box constraints)
                         - "SLSQP" (Sequential Least Squares Programming)
        entanglement (str): The entanglement strategy for the variational circuit (e.g., "linear", "full").
        layers (int): Number of layers (repetitions) in the variational quantum circuit.
        color (str): Color for the optimization progress plot (e.g., "blue", "red").

    Returns:
        dict: A dictionary containing:
              - "optimized_params" (array): The optimized parameter values.
              - "final_cost" (float): The final cost function value after optimization.
              - "cost_history" (list): A history of cost values throughout the optimization process.

    The function also plots the cost function's progress over optimization iterations.

    """
    cost_history = []

    def callback_function(params):
        """
        Track the cost value during optimization.
        """
        cost = costfunction(params, features, labels, entanglement, layers)
        cost_history.append(cost)
        print(f"Cost: {cost}")

    result = minimize(
        fun=costfunction,
        x0=params,
        args=(features, labels, entanglement, layers),
        method=optimizer,  # Supports COBYLA, L-BFGS-B, SLSQP, etc.
        callback=callback_function,
        options={"disp": True, "maxiter": 2000}
    )

    # Plot the optimization progress
    plt.figure(figsize=(8, 5))
    plt.plot(cost_history, marker="o", color=color)
    plt.xlabel("Iteration")
    plt.ylabel("Cost Function Value")
    plt.title("Optimization Progress")
    plt.grid()
    plt.show()

    return {
        "optimized_params": result.x,  # Trained parameter values
        "final_cost": result.fun,  # Final cost function value
        "cost_history": cost_history,  # Track optimization progress
    }


In [20]:
# TESTING!
# Uncomment the following code to train the model.

# X_train, X_test, y_train, y_test = train_test_split(
#     process_images("shapes.zip")[0],  # Extract and preprocess features
#     process_images("shapes.zip")[1],  # Extract corresponding target quantum states
#     test_size=0.2,  # 20% of the data is reserved for testing
#     random_state=42  # Ensures reproducibility of the split
# )

# print("Training features shape:", X_train.shape, "Training labels shape:", y_train.shape)

# # Initialize parameters. The number of parameters must match the number of parameters in TwoLocal.
# # In this example, we initialize with 64 parameters (adjust if needed based on the circuit size).
# initial_params = np.random.uniform(-np.pi, np.pi, 64)

# # Run the optimization process using the specified optimizer and circuit configuration.
# optimization(
#     cost_function,  # The cost function to minimize
#     initial_params,  # Initial variational parameters
#     X_train,  # Training feature set
#     y_train,  # Training labels (quantum states)
#     "L-BFGS-B",  # Chosen classical optimizer
#     "full",  # Entanglement strategy
#     5,  # Number of layers in the variational quantum circuit
#     "#FF1493"  # Color for the optimization plot
# )
